# Ahmad Masood
# 20I-1754
# AI-K

In [25]:
import pandas as pd
import numpy as np
import math

In [4]:
tennis_df=pd.read_csv('play_tennis.csv')
tennis_df

,day,outlook,temp,humidity,wind,play
0,D1,Sunny,Hot,High,Weak,No
1,D2,Sunny,Hot,High,Strong,No
2,D3,Overcast,Hot,High,Weak,Yes
3,D4,Rain,Mild,High,Weak,Yes
4,D5,Rain,Cool,Normal,Weak,Yes
5,D6,Rain,Cool,Normal,Strong,No
6,D7,Overcast,Cool,Normal,Strong,Yes
7,D8,Sunny,Mild,High,Weak,No
8,D9,Sunny,Cool,Normal,Weak,Yes
9,D10,Rain,Mild,Normal,Weak,Yes


In [5]:
tennis_df=tennis_df.iloc[0: ,1:,]
tennis_df

,outlook,temp,humidity,wind,play
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes
5,Rain,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rain,Mild,Normal,Weak,Yes


In [29]:
def entropy_of_y(y):
    H_y = 0
#     unique_values=np.unique(y)
#     num_unique = len(np.unique(y))
    unique_values , num_unique = np.unique(y , return_counts=True)
    
    for i in num_unique:
        H_y += i/(len(y)) * math.log2(i/(len(y)))
    return H_y*(-1)

In [57]:
def information_gain(x,y):
    x=np.array(x)
    y=np.array(y)
#     unique_values_x=np.unique(x)
#     num_unique_x = len(np.unique(x))
#     unique_values_y=np.unique(y)
#     num_unique_y = len(np.unique(y))
    unique_values_x , num_unique_x = np.unique(x , return_counts=True)
    unique_values_y , num_unique_y = np.unique(y , return_counts=True)
    H_y = entropy_of_y(y)
    
    yes_counts = []
    no_counts  = []
    
    for j in unique_values_x: 
        num_yes = 0
        num_no  = 0
         
        for k in range(len(x)):
            if j == x[k]:
                if  tennis_df['play'][k] == 'Yes':
                    num_yes +=1
                else:
                    num_no+=1
        yes_counts.append(num_yes)
        no_counts.append(num_no)
        
    H_x=0
    entropies_x=[]
    for i in range(len(unique_values_x)):
        H_x=0
        
        if (yes_counts[i]==0):
            H_x += -(no_counts[i]/(num_unique_x[i]) *  math.log2(no_counts[i]/(num_unique_x[i])))
        elif (no_counts[i]==0):
            H_x += -(yes_counts[i]/(num_unique_x[i]) *  math.log2(yes_counts[i]/(num_unique_x[i])))
            
        else:
            H_x += -(yes_counts[i]/(num_unique_x[i]) *  math.log2(yes_counts[i]/(num_unique_x[i])))
            H_x += -(no_counts[i]/(num_unique_x[i]) *  math.log2(no_counts[i]/(num_unique_x[i])))
        
        entropies_x.append(H_x)
    H_total=0
    for i in range(len(num_unique_x)):
        H_total += entropies_x[i] * (num_unique_x[i]/len(y))
    
    return H_y - H_total

### Information Gains

In [58]:
info_gain_features=[0]*4

In [59]:
info_gain_features[0] = information_gain(tennis_df['outlook'] , tennis_df['play'])
info_gain_features[1] = information_gain(tennis_df['temp'] , tennis_df['play'])
info_gain_features[2] = information_gain(tennis_df['humidity'] , tennis_df['play'])
info_gain_features[3] = information_gain(tennis_df['wind'] , tennis_df['play'])

In [67]:
col=tennis_df.columns
for i in range(len(info_gain_features)):
    print("The information Gain of ",end="")
    print(col[i], " : ",info_gain_features[i])

The information Gain of outlook  :  0.24674981977443933
The information Gain of temp  :  0.02922256565895487
The information Gain of humidity  :  0.15183550136234159
The information Gain of wind  :  0.04812703040826949


#### Splitting Data

In [81]:
outlook_df=[]
for i in tennis_df['outlook'].unique():
    outlook_df.append(tennis_df[tennis_df['outlook']==i])

temp_df=[]
for i in outlook_df[0]['temp'].unique():
    temp_df.append(outlook_df[0][outlook_df[0]['temp']==i])

wind_df=[]
for i in outlook_df[2]['wind'].unique():
    wind_df.append(outlook_df[2][outlook_df[2]['wind']==i])

In [85]:
information_gain_outlook=[0]*6
information_gain_outlook[0] = information_gain((outlook_df[0]['temp']) , (outlook_df[0]['play']))
information_gain_outlook[1] = information_gain((outlook_df[0]['humidity']) , (outlook_df[0]['play']))
information_gain_outlook[2] = information_gain((outlook_df[0]['wind']) , (outlook_df[0]['play']))

information_gain_outlook[3] = information_gain((outlook_df[2]['temp']) , (outlook_df[2]['play']))
information_gain_outlook[4] = information_gain((outlook_df[2]['humidity']) , (outlook_df[2]['play']))
information_gain_outlook[5] = information_gain((outlook_df[2]['wind']) , (outlook_df[2]['play']))

In [83]:
information_gain_temp=[0]*2
information_gain_temp[0] = information_gain((temp_df[1]['humidity']) , (temp_df[1]['play']))
information_gain_temp[1] = information_gain((temp_df[1]['wind']) , (temp_df[1]['play']))

## Predict

In [88]:
def predict (features):
    label = 0
    if features[0]=='Overcast':
        label = 'Yes'
    elif features[0] == 'Rain':
        if features[3] == 'Weak':
            label='Yes'
        elif features[3] == 'Strong':
            label='No'
    elif features[0] == 'Sunny':
        if features[2] == 'High':
            label = 'No'
        elif features[2] == 'Normal':
            label = 'Yes'
    return label

In [90]:
feat=['Overcast' , 'Hot' , 'High' , 'Weak']
pred=predict (feat)
pred

'Yes'

## Task 02

#### A classification algorithm that works well for binary and multiclass classification is called Naive Bayes. Compared to numerical input variables, naive Bayes performs better in cases of categorical input variables. It is helpful for anticipating data and making predictions based on past performance.


#### Decision trees can be applied to categorical and numerical data. You can assess your possibilities with the use of decision trees. Decision trees are great tools for assisting you in selecting one course of action over others.

### Rough Work

In [54]:
len(np.unique((np.array(tennis_df["play"]))))

2

In [20]:
y=np.array(tennis_df["play"])

In [24]:
H_y=0
unique_values=np.unique(y)
num_unique = len(np.unique(y))

for i in range(num_unique):
    H_y += i/(len(y)) * math.log2(i/(len(y)))

ValueError: math domain error

In [26]:
unique , counts = np.unique(y , return_counts=True)

In [27]:
counts

array([5, 9], dtype=int64)

In [28]:
unique

array(['No', 'Yes'], dtype=object)